In [1]:
import os
import torchaudio
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

In [2]:
# Define the custom estimator class RNNClassifier
class RNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, learning_rate, num_epochs):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs

    def fit(self, X, y):
        # Create and initialize the RNN model
        self.model = nn.Sequential(
            nn.RNN(self.input_size, self.hidden_size, self.num_layers, batch_first=True),
            nn.Linear(self.hidden_size, self.num_classes)
        )

        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        # Move model to GPU if available
        device = torch.device("cpu")
        self.model.to(device)

        # Training loop
        for epoch in range(self.num_epochs):
            self.model.train()
            for audio_batch, class_labels in X:
                optimizer.zero_grad()

                # Move audio batch to the device
                audio_batch = audio_batch.to(device)

                # Convert class labels to tensors
                class_labels = torch.tensor(class_labels, dtype=torch.long).to(device)

                outputs = self.model(audio_batch)
                loss = criterion(outputs, class_labels)
                loss.backward()
                optimizer.step()

        return self

    def predict(self, X):
        self.model.eval()
        predictions = []
        with torch.no_grad():
            for audio_batch, _ in X:
                audio_batch = audio_batch.to(device)
                outputs = self.model(audio_batch)
                _, predicted = torch.max(outputs.data, 1)
                predictions.extend(predicted.cpu().numpy())
        return predictions

In [3]:
def load_audio_dataset(folder_path):
    data = []
    labels = []

    audio_files = [file for file in os.listdir(folder_path) if file.endswith(".wav")]

    for file in tqdm(audio_files, desc=f"Loading {folder_path}"):
        audio_file = os.path.join(folder_path, file)

        # Determine the class label based on the file name
        label = 0 if file.startswith("common") else 1  # Assuming 0 for "Pristine" and 1 for "Altered"

        waveform, sample_rate = torchaudio.load(audio_file)

        data.append(waveform)
        labels.append(label)

    return data, labels



In [4]:
data_root = "C:\\Users\\User\\Apviza\\Project 6\\Truncated"

batch_size = 32

# Load data from the train, test, and validation folders
train_folder = os.path.join(data_root, "train")
test_folder = os.path.join(data_root, "test")
val_folder = os.path.join(data_root, "validation")

train_data, train_labels = load_audio_dataset(train_folder)
test_data, test_labels = load_audio_dataset(test_folder)
val_data, val_labels = load_audio_dataset(val_folder)

# Create DataLoader instances
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(list(zip(test_data, test_labels)), batch_size=batch_size, shuffle=False)
val_loader = DataLoader(list(zip(val_data, val_labels)), batch_size=batch_size, shuffle=False)

Loading C:\Users\User\Apviza\Project 6\Truncated\train: 100%|██████████| 33212/33212 [00:30<00:00, 1078.38it/s]
Loading C:\Users\User\Apviza\Project 6\Truncated\test: 100%|██████████| 7057/7057 [00:06<00:00, 1088.74it/s]
Loading C:\Users\User\Apviza\Project 6\Truncated\validation: 100%|██████████| 6892/6892 [00:06<00:00, 1060.41it/s]


In [8]:
# Define hyperparameters
input_size = 23296  
hidden_size = 64
num_classes = 2  # Pristine and Altered
num_layers = 2
learning_rate = 0.001
num_epochs = 10

In [9]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Take the output of the last time step
        return out

In [10]:
model = RNNModel(input_size, hidden_size, num_layers, num_classes)


In [11]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
device = torch.device("cpu")
model.to(device)

RNNModel(
  (rnn): RNN(23296, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)

In [20]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for audio_batch, class_labels in train_loader:
        optimizer.zero_grad()

        # Move audio batch to the device
        audio_batch = pad_sequence(audio_batch, batch_first=True).to(device)

        # Convert class labels to tensors
        class_labels = class_labels.to(device)  # No need to convert to LongTensor here

        outputs = model(audio_batch)
        loss = criterion(outputs, class_labels)
        loss.backward()
        optimizer.step()

        # Track statistics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == class_labels).sum().item()
        total_samples += class_labels.size(0)

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for audio_batch, class_labels in val_loader:
            # Move audio batch to the device
            audio_batch = pad_sequence(audio_batch, batch_first=True).to(device)

            class_labels = class_labels.to(device)  # No need to convert to LongTensor here

            outputs = model(audio_batch)
            loss = criterion(outputs, class_labels)

            # Track validation statistics
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == class_labels).sum().item()
            val_total += class_labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = (total_correct / total_samples) * 100

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = (val_correct / val_total) * 100

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# After training is complete, you can evaluate the model on the test data if needed
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for audio_batch, class_labels in test_loader:
        # Move audio batch to the device
        audio_batch = pad_sequence(audio_batch, batch_first=True).to(device)

        class_labels = class_labels.to(device)  # No need to convert to LongTensor here

        outputs = model(audio_batch)
        _, predicted = torch.max(outputs.data, 1)
        test_total += class_labels.size(0)
        test_correct += (predicted == class_labels).sum().item()

test_accuracy = (test_correct / test_total) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/10, Train Loss: 0.0073, Train Accuracy: 99.76%, Validation Loss: 0.1686, Validation Accuracy: 96.37%
Epoch 2/10, Train Loss: 0.0058, Train Accuracy: 99.80%, Validation Loss: 0.1988, Validation Accuracy: 96.30%
Epoch 3/10, Train Loss: 0.0045, Train Accuracy: 99.87%, Validation Loss: 0.1801, Validation Accuracy: 96.39%
Epoch 4/10, Train Loss: 0.0065, Train Accuracy: 99.77%, Validation Loss: 0.2111, Validation Accuracy: 96.34%
Epoch 5/10, Train Loss: 0.0031, Train Accuracy: 99.91%, Validation Loss: 0.2205, Validation Accuracy: 96.50%
Epoch 6/10, Train Loss: 0.0019, Train Accuracy: 99.94%, Validation Loss: 0.2668, Validation Accuracy: 96.18%
Epoch 7/10, Train Loss: 0.0039, Train Accuracy: 99.87%, Validation Loss: 0.2077, Validation Accuracy: 96.34%
Epoch 8/10, Train Loss: 0.0030, Train Accuracy: 99.89%, Validation Loss: 0.2472, Validation Accuracy: 96.07%
Epoch 9/10, Train Loss: 0.0019, Train Accuracy: 99.92%, Validation Loss: 0.2107, Validation Accuracy: 96.65%
Epoch 10/10, Train 